# Preprocessin for Sentiment Analysis
> * [공공데이터포털](https://www.data.go.kr/) : 뉴스빅데이터 분석 정보
> * 수정일 2017.12.26 

In [69]:
# -*- coding: utf-8 -*-

In [70]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import re

In [71]:
path1 = './data/news/have_negative_positive/constitution/'
path2 = './data/news/have_negative_positive/household_debt/'
path3 = './data/news/have_negative_positive/olymphic/'
path4 = './data/news/have_negative_positive/'
pathList = [path1, path2, path3, path4]

In [72]:
def Get_CSVfileList(path):
    filelist = glob(path+'*.csv')
    filelist.extend(glob(path+'*.CSV'))
    return filelist

In [73]:
df_data=pd.DataFrame()
x = 0 
for path in pathList:
    fileList = Get_CSVfileList(path)
    for file in fileList:
        df = pd.read_csv(file,encoding='utf-8', engine='python')
        x += df.shape[0]
        if len(df_data) == 0:
            df_data = df[['감성분석', '긍정 문장', '부정 문장']]
        else:
            df_data = pd.concat([df_data, df[['감성분석', '긍정 문장', '부정 문장']]])
df_data.reset_index(drop=True, inplace=True)        
print (x)
print (df_data.shape)
df_data.head()

353286
(353286, 3)


,감성분석,긍정 문장,부정 문장
0,NEGATIVE 고통 NEGATIVE 좌절 NEGATIVE 갈등 NEGATIVE 불...,이젠 민주개혁의 길로(사설) 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,우리는 과연 과거의 온갖 간난과 고통을 극복하고 새 시대에 들어섰는가. 이젠 갈등과...
1,NEGATIVE 가하 NEGATIVE 배제 NEGATIVE 까지 NEGATIVE 문...,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,여야는 특히 금년 상반기중에 실시토록 돼 있는 지방선거에서 연합공천의 길이 열림에 ...
2,NEGATIVE 힘들 NEGATIVE 동상이몽 NEGATIVE 힘들 NEGATIVE...,"4당,지자제 연합공천 대비 「파트너 탐색전」 이같은 구상을 실현시키기 위해 민정당은...",또 연합공천의 길이 열려 있는 지방선거를 앞두고 각 정당간에 구체적인 막후협상이 활...
3,NEGATIVE 배제 NEGATIVE 미달 NEGATIVE 모순 NEGATIVE 약하,NaN,민정당은 일단 평민ㆍ민주ㆍ공화 등 기존 모든 야당과의 정책제휴및 정당연합 가능성을 ...
4,POSITIVE 선진국 POSITIVE 극복 NEGATIVE 파산 NEGATIVE ...,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,세대교체가 될 가능성은 없습니까. 3김은 이미 도덕적 파산선고를 받은 사람들 아닙니...


In [74]:
df_words = df_data['감성분석'].copy()
df_positive = df_data['긍정 문장'].copy()
df_negative = df_data['부정 문장'].copy()

* drop list
> 기자 & 이메일 주소만 있는 문장  
> Neutral 제거

## 긍정 문장 추출

### NaN & ' '제거

In [75]:
df_positive.dropna(inplace = True)
df_positive = df_positive.apply(html.unescape)
df_positive.reset_index(drop=True, inplace = True)
df_positive.drop(df_positive[df_positive.str.match('^ $')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(312378,)


### #*****? 인 값 제거

In [76]:
df_positive.drop(df_positive[df_positive.str.match('^#[a-zA-Z0-9]+\?$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(312232,)


### #*****! 인 값 제거

In [77]:
df_positive.drop(df_positive[df_positive.str.match('^#[a-zA-Z0-9]+\!$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(312178,)


### 이메일만 있는 값 제거

In [78]:
df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')].index,inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(310534,)


### 이메일과 기자명만 있는 값 제거

In [79]:
df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index , inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-. ]*$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[가-힣]{3,3}기자')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[. ][가-힣]{3,3}기자')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[가-힣 ]{3,4}기자$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_. ]+[가-힣 ]{3,4}기자$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+@$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+@([a-zA-Z0-9-]+\.[a-zA-Z0-9-.])*')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('.[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[가-힣+_.=]+?[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.len() < 11].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[(df_positive.str.match('^[가-힣]{3,5}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\| @]*?')) & (df_positive.str.len() < 100)].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

print (df_positive[df_positive==''])

df_positive = df_positive.apply(html.unescape)

df_positive.drop_duplicates(inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)
df_positive = df_positive.apply(html.unescape)
df_positive.drop_duplicates(inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(309106,)
(308258,)
(308204,)
(308164,)
(308104,)
(308070,)
(307538,)
(307524,)
(307310,)
(307000,)
(306200,)
(305706,)
(302802,)
(302544,)
Series([], Name: 긍정 문장, dtype: object)
(144397,)
(144397,)


In [80]:
df_positive.to_csv('./data/sentiment_data/positive_sentiment_sentence_from_news.csv', index=False, header= False, encoding='utf-8')

## 부정 문장 추출

### NaN & ' '제거

In [81]:
df_negative.dropna(inplace = True)
df_negative = df_negative.apply(html.unescape)
df_negative.reset_index(drop=True, inplace = True)
df_negative.drop(df_negative[df_negative.str.match('^ $')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(321744,)


### #*****? 인 값 제거

In [82]:
df_negative.drop(df_negative[df_negative.str.match('^#[a-zA-Z0-9]+\?$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(321664,)


### #*****! 인 값 제거

In [83]:
df_negative.drop(df_negative[df_negative.str.match('^#[a-zA-Z0-9]+\!$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(321664,)


### 이메일만 있는 값 제거

In [84]:
df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')].index,inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(321664,)


### 이메일과 기자명만 있는 값 제거

In [85]:
df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index , inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-. ]*$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[가-힣]{3,3}기자')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[. ][가-힣]{3,3}기자')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[가-힣 ]{3,4}기자$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_. ]+[가-힣 ]{3,4}기자$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+@$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+@([a-zA-Z0-9-]+\.[a-zA-Z0-9-.])*')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('.[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[가-힣+_.=]+?[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.len() < 11].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[(df_negative.str.match('^[가-힣]{3,5}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\| @]*?')) & (df_negative.str.len() < 100)].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

print (df_negative[df_negative==''])

df_negative = df_negative.apply(html.unescape)

df_negative.drop_duplicates(inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)
df_negative = df_negative.apply(html.unescape)
df_negative.drop_duplicates(inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(321224,)
(321116,)
(321086,)
(321072,)
(321020,)
(320454,)
(320280,)
(320280,)
(319886,)
(319726,)
(319398,)
(319000,)
(316976,)
(316652,)
Series([], Name: 부정 문장, dtype: object)
(151957,)
(151957,)


In [86]:
df_negative.to_csv('./data/sentiment_data/negative_sentiment_sentence_from_news.csv', index=False, header=False, encoding='utf-8')

## 긍정 / 부정 단어 추출

In [87]:
df_words.dropna(inplace = True)
df_words = df_words.apply(html.unescape)
df_words.reset_index(drop = True, inplace = True)
df_words.drop(df_words[df_words.str.match('^ $')].index, inplace = True)
df_words.reset_index(drop = True, inplace = True)
df_words = df_words.apply(html.unescape)
print(df_words.shape)

(334174,)


In [88]:
df_words = df_words[df_words.str.match('NEGATIVE|POSITIVE')]
df_words.reset_index(drop = True, inplace = True)
print (df_words.shape)

(328092,)


In [89]:
negativeWords = df_words.str.findall('(NEGATIVE [가-힣]+)').str.join(' ')
negativeWords.dropna(inplace = True)
negativeWords.reset_index(drop=True, inplace = True)
positiveWords = df_words.str.findall('(POSITIVE [가-힣]+)').str.join(' ')
positiveWords.dropna(inplace = True)
positiveWords.reset_index(drop=True, inplace = True)
print (negativeWords.shape, positiveWords.shape)

(328092,) (328092,)


In [90]:
positiveWords.drop(positiveWords[positiveWords==''].index, inplace = True)
positiveWords.reset_index(drop=True, inplace = True)
negativeWords.drop(negativeWords[negativeWords==''].index, inplace = True)
negativeWords.reset_index(drop=True, inplace = True)
print (negativeWords.shape, positiveWords.shape)
positiveWords = positiveWords.str.findall('([가-힣]+)')
negativeWords = negativeWords.str.findall('([가-힣]+)')

(288850,) (247246,)


In [91]:
positiveWords = list(set(itertools.chain.from_iterable(positiveWords)))
negativeWords = list(set(itertools.chain.from_iterable(negativeWords)))
print (len(positiveWords), len(negativeWords))

5010 7608


In [92]:
positiveWords = pd.Series(positiveWords)
negativeWords = pd.Series(negativeWords)

In [93]:
positiveWords.to_csv('./data/sentiment_data/positive_sentiment_word_from_news.csv', index=False, header=False)
negativeWords.to_csv('./data/sentiment_data/negative_sentiment_word_from_news.csv', index=False, header=False)

In [94]:
df_positive = pd.concat([df_positive, pd.DataFrame(len(df_positive)*[1], columns=[1])],axis = 1)
df_positive.rename(columns={'긍정 문장':0},inplace = True)
df_negative = pd.concat([df_negative, pd.DataFrame(len(df_negative)*[0], columns=[1])],axis = 1)
df_negative.rename(columns={'부정 문장':0},inplace = True)


In [95]:
newsSentence = pd.concat([df_positive, df_negative], axis = 0)
newsSentence.reset_index(drop=True, inplace = True)
print (newsSentence.shape)
newsSentence.head()

(296354, 2)


,0,1
0,이젠 민주개혁의 길로(사설) 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 「파트너 탐색전」 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표,「지난시대」의 완전청산을 선언한 후...",1


## 기자명 & 이메일 제거 및 50자 이내 제거

In [96]:
newsSentence[0] = newsSentence[0].str.replace('^[가-힣]{3,4}[//. ]?기자[//. ]',' ')

newsSentence[0] = newsSentence[0].str.replace('^[ ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]*',' ')

newsSentence[0] = newsSentence[0].str.replace('[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',' ')
print (newsSentence.shape)

newsSentence[0] = newsSentence[0].str.replace('[가-힣]{3,4}[//. ]?기자',' ')

newsPress = pd.read_csv('./data/newspress.txt',encoding='utf-8',header=None)
newsPress.head()

for idx in newsPress.index:
    newsSentence[0] = newsSentence[0].str.replace(newsPress.loc[idx][0],' ')

newsSentence[0] = newsSentence[0].str.replace('[☜\[\]☞※©◀◆◁◀◈○◌◎●◑◐☆★\|♠♡〈〉『」「』▲=⑨ ①② ③「」◎○~!@#$%^&*()_+|<>?:{}]', ' ')

newsSentence[0] = newsSentence[0].str.replace('[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]*','')

newsSentence[0] = newsSentence[0].str.replace('[a-zA-Z0-9+-.]+@','')

newsSentence[0] = newsSentence[0].str.strip()

newsSentence.drop(newsSentence[newsSentence[0].str.len()<10].index, inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.dropna(inplace=True)
print (newsSentence.shape)

newsSentence.drop(newsSentence[newsSentence[0].str.len()<50].index, inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.dropna(inplace=True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.drop_duplicates(inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
print (newsSentence.shape)
newsSentence.to_csv('./data/sentiment_data/merged_sentiment_data.txt',index=None, header=None)
newsSentence.head()

(296354, 2)
(295847, 2)
(278959, 2)


,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1
